In [1]:
import pandas as pd
import numpy as np

import networkx as nx
from networkx.algorithms.community import k_clique_communities
import random as rd
rd.seed(42)

import warnings
warnings.filterwarnings('ignore')

## Random Walk Sampling Method 2
### Sample 1

In [2]:
# Using first sample
G_rwm2_1 = nx.read_gml(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\Samples\Random Walk Sampling\G_rw2_1.gml')

In [3]:
# Confirm that Graph is Directed
nx.is_directed(G_rwm2_1)

True

In [4]:
# Display Size of the Graph
print(nx.info(G_rwm2_1))

MultiDiGraph with 50000 nodes and 137944 edges


Confirmed that the graph meets the criterea of having more edges than nodes and maintained the directed structure.

In [5]:
# List all nodes in the sample
n_rwm2_1 = list(G_rwm2_1.nodes)

In [6]:
# Get node properties from data exploration exercise for comparison
node_properties = pd.read_csv(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\node_properties_cl.csv')
node_properties[['degree', 'in_degree', 'out_degree', 'eigen_centrality']].describe()

,degree,in_degree,out_degree,eigen_centrality
count,4.666873e+06,4.666873e+06,4.666873e+06,4.666873e+06
mean,4.494298e+00,2.247149e+00,2.247149e+00,3.970074e-05
std,3.935922e+01,1.998412e+01,2.868249e+01,4.611943e-04
min,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
25%,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
50%,2.000000e+00,1.000000e+00,1.000000e+00,4.892620e-20
75%,3.000000e+00,1.000000e+00,2.000000e+00,1.517475e-14
max,2.791700e+04,1.211600e+04,2.533500e+04,1.642653e-01


In [7]:
rwm2_1_df = nx.to_pandas_edgelist(G_rwm2_1) 
rwm2_1_df.head()

,source,target,weight
0,18WtUF8esLdNXXCuEmourMFafpcwTunUZX,18v8u1VLQtA4RVTq4zjLJLjA3TV86DVn9A,1.0
1,18WtUF8esLdNXXCuEmourMFafpcwTunUZX,39j7zut99kuEhA28kfW6jv2cHc7jhrWY3c,1.0
2,1MuxrCQ6T6Ng9445SE8CxqnGcZK9i9eefa,3KSk127k2cSoesFj4YbqqqTqGTB6AMqBDS,1.0
3,19ccn7Wj1hATVL6nktupZu6hZwez6rCS8f,1AjCcmjLeoz5KpFJRkrhjtJ82NQAWYFsmg,1.0
4,19ccn7Wj1hATVL6nktupZu6hZwez6rCS8f,1GQvGPK6ab33EfjbXABXBRfshBseV4uhXi,1.0


In [8]:
# Number of unique input addresses
rwm2_1_df['source'].nunique()

32154

In [9]:
# Number of unique output addresses
rwm2_1_df['target'].nunique()

31367

We observe relative balance between input and output addresses, with slightly more input addresses. In the full dataset there is also relative balance, but there were more output addresses. 

In [10]:
rwm2_1_df.describe()

,weight
count,137944.000000
mean,0.774923
std,0.468702
min,-69.648093
25%,0.507070
50%,1.000000
75%,1.000000
max,52.821905


The weight variable shows that the distribution of the weighting (amount*0.5 + timestamp*0.5) is similar to that of overall transaction amounts.

In [11]:
# Subset by fraudulent/high-risk nodes
fr_nodes = node_properties[node_properties['fraud_flag'] == 1]
fr_nodes = fr_nodes['address'].tolist()
print('Total number of fraudulent/high-risk nodes : ', len(fr_nodes))

# Check for number of fraudulent/high-risk addresses in the sample
print('Number of fraudulent/high-risk nodes in sample : ', len(set(n_rwm2_1).intersection(fr_nodes)))

Total number of fraudulent/high-risk nodes :  4563
Number of fraudulent/high-risk nodes in sample :  280


These numbers suggest that random selection of nodes worsens the class imbalance problem, but when considering the counts relative to the size of the network the issue is less pronounced.

Full Network: 0.11% 

Sample: 0.56%

## Node Properties

In [12]:
# Get Properties

address = [node for (node, val) in G_rwm2_1.degree()]
degree = [val for (node, val) in G_rwm2_1.degree()]
in_degree = [val for (node, val) in G_rwm2_1.in_degree()]
out_degree = [val for (node, val) in G_rwm2_1.out_degree()]
#eigen_centrality = nx.eigenvector_centrality(G_rwm2_1) # NetworkXNotImplemented: not implemented for multigraph type
#eigen_centrality = [eigen_centrality[node] for node in eigen_centrality]

In [13]:
# Add Properties to DataFrame

nodes = pd.DataFrame()
nodes['address'] = address
nodes['degree'] = degree
nodes['in_degree'] = in_degree
nodes['out_degree'] = out_degree
#nodes['eigen_centrality'] = eigen_centrality

In [14]:
# Add fraud flag

nodes = nodes.assign(**dict.fromkeys(['fraud_flag'], 0))

for i in fr_nodes:
    nodes.loc[nodes.address == i, 'fraud_flag'] = 1
    
nodes.head()

,address,degree,in_degree,out_degree,fraud_flag
0,18WtUF8esLdNXXCuEmourMFafpcwTunUZX,3,1,2,0
1,1MuxrCQ6T6Ng9445SE8CxqnGcZK9i9eefa,1,0,1,0
2,1Ke6qBAEEjZEGjQ3b9NEB1aqFeiVgJdAsd,2,2,0,0
3,19ccn7Wj1hATVL6nktupZu6hZwez6rCS8f,6,3,3,0
4,12LJeywVN3UFefbFy53HLJL6FAFet3SWAA,1,0,1,0


In [15]:
# Describe sample properties overall

nodes[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,50000.00000,50000.000000,50000.000000
mean,5.51776,2.758880,2.758880
std,39.43392,17.452118,25.808109
min,1.00000,0.000000,0.000000
25%,1.00000,0.000000,0.000000
50%,2.00000,1.000000,1.000000
75%,4.00000,2.000000,2.000000
max,4175.00000,1486.000000,2689.000000


In [16]:
# Describe Properties of Fraudulent Nodes

nodes_fraud = nodes[nodes['fraud_flag'] == 1]
nodes_fraud[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,280.000000,280.000000,280.000000
mean,138.235714,56.203571,82.032143
std,468.777065,186.206955,312.677194
min,1.000000,0.000000,0.000000
25%,5.000000,2.000000,1.000000
50%,29.500000,9.500000,8.000000
75%,78.000000,38.250000,33.000000
max,4175.000000,1486.000000,2689.000000


In [17]:
# Describe Properties of Non-Fraudulent Nodes

nodes_licit = nodes[nodes['fraud_flag'] == 0]
nodes_licit[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,49720.000000,49720.000000,49720.000000
mean,4.770354,2.457904,2.312450
std,15.195082,9.774776,9.251823
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,2.000000,1.000000,1.000000
75%,4.000000,2.000000,2.000000
max,958.000000,954.000000,725.000000


For all 3 cases the degree distribution properties are similar to the overall network.

## Graph Properties

In [18]:
# Density
nx.density(G_rwm2_1) # Connectivity remains very low

5.517870357407148e-05

In [19]:
# Communities - k-Cliques

com_generator_rwm2_1 = k_clique_communities(G_rwm2_1.to_undirected(), k=10)
com_rwm2_1 = next(com_generator_rwm2_1)
print('The number of communities in the sample : ' + str(len(com_rwm2_1)))

The number of communities in the sample : 91


The sample has a large variety of community structures. (Note: Giant Component = 107)